# SparkContext

In [1]:
from pyspark import SparkConf, SparkContext 
# https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.SparkContext.html

spark_conf = SparkConf().setAppName("spark-context-test")
spark = SparkContext(conf = spark_conf)
SparkConf().getAll()

24/07/13 17:23:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


[('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.submit.deployMode', 'client'),
 ('spark.app.name', 'spark-context-test'),
 ('spark.executor.extraJavaOptions',
  '-Dio.netty.tryReflectionSetAccessible=true'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.driver.extraJavaOptions',
  '-Dio.netty.tryReflectionSetAccessible=true')]

# SparkSession

In [2]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
        .master("local")
        .appName("spark-session-test")
#         .config("spark.some.config.option", "some-value")
        .getOrCreate()
)
print(f"Hello, {spark.sparkContext.sparkUser()}")

# enableHiveSupport()

Hello, jovyan


# WordCount

In [27]:
spark = (
    SparkSession.builder
        .master("local")
        .appName("word-count")
        .getOrCreate()
)
sc = spark.sparkContext

logFile = "/home/jovyan/data/2024-05-23-15.json"
logData = sc.textFile(logFile).cache()

numAs = logData.filter(lambda s: 'a' in s).count()
numBs = logData.filter(lambda s: 'b' in s).count()

print(f"Lines with a: {numAs}, lines with b: {numBs}")

Lines with a: 253551, lines with b: 253551


In [31]:
import yfinance as yf

tsla = yf.Ticker("TSLA")
# print(f"ticker: {tsla}")
# print(f"history_metadata \n: {tsla.history_metadata}")

hist = tsla.history(period="1y")
# print(f"history type: {type(hist)}")
# print(f"history: \n{hist}")

df = spark.createDataFrame(hist)
df.printSchema()

print(df.count())

df.show(1, False)

root
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: long (nullable = true)
 |-- Dividends: double (nullable = true)
 |-- Stock Splits: double (nullable = true)

252
+-----------------+------------------+-----------------+-----------------+---------+---------+------------+
|Open             |High              |Low              |Close            |Volume   |Dividends|Stock Splits|
+-----------------+------------------+-----------------+-----------------+---------+---------+------------+
|274.5899963378906|279.45001220703125|270.6000061035156|277.8999938964844|112681500|0.0      |0.0         |
+-----------------+------------------+-----------------+-----------------+---------+---------+------------+
only showing top 1 row



# Read Data

In [26]:
df = spark.read.json("../data/2024-05-23-15.json")
df.printSchema()

root
 |-- actor: struct (nullable = true)
 |    |-- avatar_url: string (nullable = true)
 |    |-- display_login: string (nullable = true)
 |    |-- gravatar_id: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- login: string (nullable = true)
 |    |-- url: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- org: struct (nullable = true)
 |    |-- avatar_url: string (nullable = true)
 |    |-- gravatar_id: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- login: string (nullable = true)
 |    |-- url: string (nullable = true)
 |-- payload: struct (nullable = true)
 |    |-- action: string (nullable = true)
 |    |-- before: string (nullable = true)
 |    |-- comment: struct (nullable = true)
 |    |    |-- _links: struct (nullable = true)
 |    |    |    |-- html: struct (nullable = true)
 |    |    |    |    |-- href: string (nullable = true)
 |    |    |    |-- pull_request: struct (nul

24/07/13 17:46:09 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [28]:
df.count()

253551

In [29]:
df.show(1, False)

+---------------------------------------------------------------------------------------------------------------------------------+--------------------+-----------+-----------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [32]:
df = spark.read.format('org.apache.spark.sql.json') \
        .load("../data/2024-05-23-15.json")
df.show(3)

+--------------------+--------------------+-----------+--------------------+--------------------+------+--------------------+--------------------+
|               actor|          created_at|         id|                 org|             payload|public|                repo|                type|
+--------------------+--------------------+-----------+--------------------+--------------------+------+--------------------+--------------------+
|{https://avatars....|2024-05-23T15:00:00Z|38650027946|{https://avatars....|{created, null, n...|  true|{67203815, nextcl...|PullRequestReview...|
|{https://avatars....|2024-05-23T15:00:00Z|38650027982|                null|{null, 329a164168...|  true|{791862430, Claud...|           PushEvent|
|{https://avatars....|2024-05-23T15:00:00Z|38650027993|{https://avatars....|{created, null, n...|  true|{143040428, githu...|PullRequestReview...|
+--------------------+--------------------+-----------+--------------------+--------------------+------+--------------

In [ ]:
# df2 = spark.read.json(
#     ['../data/2024-05-19-14.json','../data/2024-05-19-13.json'])
# df2.show(3)  

In [33]:
df.dtypes

[('actor',
  'struct<avatar_url:string,display_login:string,gravatar_id:string,id:bigint,login:string,url:string>'),
 ('created_at', 'string'),
 ('id', 'string'),
 ('org',
  'struct<avatar_url:string,gravatar_id:string,id:bigint,login:string,url:string>'),
 ('payload',
  'struct<action:string,before:string,comment:struct<_links:struct<html:struct<href:string>,pull_request:struct<href:string>,self:struct<href:string>>,author_association:string,body:string,commit_id:string,created_at:string,diff_hunk:string,html_url:string,id:bigint,in_reply_to_id:bigint,issue_url:string,line:bigint,node_id:string,original_commit_id:string,original_line:bigint,original_position:bigint,original_start_line:bigint,path:string,performed_via_github_app:struct<created_at:string,description:string,events:array<string>,external_url:string,html_url:string,id:bigint,name:string,node_id:string,owner:struct<avatar_url:string,events_url:string,followers_url:string,following_url:string,gists_url:string,gravatar_id:str

In [34]:
from pyspark.sql.types import StructType, StructField, StringType, DateType, LongType, BooleanType
# https://spark.apache.org/docs/latest/sql-ref-datatypes.html

schema = StructType([
      StructField("actor", StringType(), True),
      StructField("created_at", DateType(), True),
      StructField("id", LongType(), False),
      StructField("org", StringType(), True),
      StructField("payload", StringType(), True),
      StructField("public", BooleanType(), True),
      StructField("repo", StringType(), True),
      StructField("type", StringType(), True)
  ])


In [36]:
df = spark.read.schema(schema).json("../data/2024-05-23-15.json")
df.printSchema()

root
 |-- actor: string (nullable = true)
 |-- created_at: date (nullable = true)
 |-- id: long (nullable = true)
 |-- org: string (nullable = true)
 |-- payload: string (nullable = true)
 |-- public: boolean (nullable = true)
 |-- repo: string (nullable = true)
 |-- type: string (nullable = true)



In [37]:
rdd = sc.textFile('../data/2024-05-23-15.json')
rdd.take(1)

['{"id":"38650027946","type":"PullRequestReviewEvent","actor":{"id":1283854,"login":"icewind1991","display_login":"icewind1991","gravatar_id":"","url":"https://api.github.com/users/icewind1991","avatar_url":"https://avatars.githubusercontent.com/u/1283854?"},"repo":{"id":67203815,"name":"nextcloud/files_antivirus","url":"https://api.github.com/repos/nextcloud/files_antivirus"},"payload":{"action":"created","review":{"id":2074246787,"node_id":"PRR_kwDOBAFy5857on6D","user":{"login":"icewind1991","id":1283854,"node_id":"MDQ6VXNlcjEyODM4NTQ=","avatar_url":"https://avatars.githubusercontent.com/u/1283854?v=4","gravatar_id":"","url":"https://api.github.com/users/icewind1991","html_url":"https://github.com/icewind1991","followers_url":"https://api.github.com/users/icewind1991/followers","following_url":"https://api.github.com/users/icewind1991/following{/other_user}","gists_url":"https://api.github.com/users/icewind1991/gists{/gist_id}","starred_url":"https://api.github.com/users/icewind1991/

# Write Data

In [38]:
df.select('type').distinct().collect()

[Row(type='PullRequestReviewEvent'),
 Row(type='PushEvent'),
 Row(type='GollumEvent'),
 Row(type='ReleaseEvent'),
 Row(type='CommitCommentEvent'),
 Row(type='CreateEvent'),
 Row(type='PullRequestReviewCommentEvent'),
 Row(type='IssueCommentEvent'),
 Row(type='DeleteEvent'),
 Row(type='IssuesEvent'),
 Row(type='ForkEvent'),
 Row(type='PublicEvent'),
 Row(type='MemberEvent'),
 Row(type='WatchEvent'),
 Row(type='PullRequestEvent')]

In [39]:
df2 = df.filter(df.type == 'CommitCommentEvent')
df2.count()

303

In [40]:
df2.write \
  .option("compression", "gzip") \
  .mode('Overwrite') \
  .json("../data/output.json")

----------------------------------------                                        
Exception occurred during processing of request from ('127.0.0.1', 47256)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.9/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.9/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.9/socketserver.py", line 747, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 262, in handle
    poll(accum_updates)
  File "/usr/local/spark/python/pyspark/accumulators.py", line 235, in poll
    if func():
  File "/usr/local/spark/python/pyspark/accumulators.py", line 239, in accum_updates
    num_updates = read_int(self.rfi

In [ ]:
# docker exec -it de-2024-spark-master-1 s park-submit --master spark://spark-master:7077 jobs/hello-world.py data/2024-05-19-14.json